In [ ]:
%pip install --upgrade langchain langchain-experimental langchain-community pyvis json-repair

In [12]:
#LLM Graph Transformer using gemma3 for all the examples
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain_community.llms import Ollama

llm = Ollama(model='gemma3', temperature=0)
graph_transformer = LLMGraphTransformer(llm=llm)

In [13]:
#Extract graph data
text = """
Albert Einstein[a] (14 March 1879 – 18 April 1955) was a German-born theoretical physicist who is best known for developing the theory of relativity. Einstein also made important contributions to quantum mechanics.[1][5] His mass–energy equivalence formula E = mc2, which arises from special relativity, has been called "the world's most famous equation".[6] He received the 1921 Nobel Prize in Physics for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect.[7]

Born in the German Empire, Einstein moved to Switzerland in 1895, forsaking his German citizenship (as a subject of the Kingdom of Württemberg)[note 1] the following year. In 1897, at the age of seventeen, he enrolled in the mathematics and physics teaching diploma program at the Swiss federal polytechnic school in Zurich, graduating in 1900. He acquired Swiss citizenship a year later, which he kept for the rest of his life, and afterwards secured a permanent position at the Swiss Patent Office in Bern. In 1905, he submitted a successful PhD dissertation to the University of Zurich. In 1914, he moved to Berlin to join the Prussian Academy of Sciences and the Humboldt University of Berlin, becoming director of the Kaiser Wilhelm Institute for Physics in 1917; he also became a German citizen again, this time as a subject of the Kingdom of Prussia.[note 1] In 1933, while Einstein was visiting the United States, Adolf Hitler came to power in Germany. Horrified by the Nazi persecution of his fellow Jews,[8] he decided to remain in the US, and was granted American citizenship in 1940.[9] On the eve of World War II, he endorsed a letter to President Franklin D. Roosevelt alerting him to the potential German nuclear weapons program and recommending that the US begin similar research.

In 1905, sometimes described as his annus mirabilis (miracle year), he published four groundbreaking papers.[10] In them, he outlined a theory of the photoelectric effect, explained Brownian motion, introduced his special theory of relativity, and demonstrated that if the special theory is correct, mass and energy are equivalent to each other. In 1915, he proposed a general theory of relativity that extended his system of mechanics to incorporate gravitation. A cosmological paper that he published the following year laid out the implications of general relativity for the modeling of the structure and evolution of the universe as a whole.[11][12] In 1917, Einstein wrote a paper which introduced the concepts of spontaneous emission and stimulated emission, the latter of which is the core mechanism behind the laser and maser, and which contained a trove of information that would be beneficial to developments in physics later on, such as quantum electrodynamics and quantum optics.[13]

In the middle part of his career, Einstein made important contributions to statistical mechanics and quantum theory. Especially notable was his work on the quantum physics of radiation, in which light consists of particles, subsequently called photons. With physicist Satyendra Nath Bose, he laid the groundwork for Bose–Einstein statistics. For much of the last phase of his academic life, Einstein worked on two endeavors that ultimately proved unsuccessful. First, he advocated against quantum theory's introduction of fundamental randomness into science's picture of the world, objecting that God does not play dice.[14] Second, he attempted to devise a unified field theory by generalizing his geometric theory of gravitation to include electromagnetism. As a result, he became increasingly isolated from mainstream modern physics.
"""


In [14]:
documents = [Document(page_content=text)]
graph_documents = await graph_transformer.aconvert_to_graph_documents(documents)

In [15]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='theory of relativity', type='Theory', properties={}), Node(id='Nobel Prize in Physics', type='Award', properties={}), Node(id='general theory of relativity', type='Theory', properties={}), Node(id='Kingdom of Württemberg', type='Government', properties={}), Node(id='Switzerland', type='Country', properties={}), Node(id='quantum electrodynamics and quantum optics', type='Fields of Study', properties={}), Node(id='two endeavors that ultimately proved unsuccessful', type='Endeavors', properties={}), Node(id='geometric theory of gravitation', type='Theory', properties={}), Node(id='four groundbreaking papers', type='Papers', properties={}), Node(id='Albert Einstein', type='Person', properties={}), Node(id='Bose–Einstein statistics', type='Statistical Model', properties={}), Node(id='German nuclear weapons program', type='Program', properties={}), Node(id="introduction of fundamental randomness into science's picture of the world", type='Concept', properties={}), Node(id='qu

In [17]:
#Visualize graph
import os
from pyvis.network import Network

def visualize_graph(graph_documents):

    # Create network
    net = Network(height="1200px", width="100%", directed=True,
                      notebook=False, bgcolor="#222222", font_color="white")
    
    nodes = graph_documents[0].nodes
    relationships = graph_documents[0].relationships

    # Build lookup for valid nodes
    node_dict = {node.id: node for node in nodes}
    
    # Filter out invalid edges and collect valid node IDs
    valid_edges = []
    valid_node_ids = set()
    for rel in relationships:
        if rel.source.id in node_dict and rel.target.id in node_dict:
            valid_edges.append(rel)
            valid_node_ids.update([rel.source.id, rel.target.id])


    # Track which nodes are part of any relationship
    connected_node_ids = set()
    for rel in relationships:
        connected_node_ids.add(rel.source.id)
        connected_node_ids.add(rel.target.id)

    # Add valid nodes
    for node_id in valid_node_ids:
        node = node_dict[node_id]
        try:
            net.add_node(node.id, label=node.id, title=node.type, group=node.type)
        except:
            continue  # skip if error

    # Add valid edges
    for rel in valid_edges:
        try:
            net.add_edge(rel.source.id, rel.target.id, label=rel.type.lower())
        except:
            continue  # skip if error

    # Configure physics
    net.set_options("""
            {
                "physics": {
                    "forceAtlas2Based": {
                        "gravitationalConstant": -100,
                        "centralGravity": 0.01,
                        "springLength": 200,
                        "springConstant": 0.08
                    },
                    "minVelocity": 0.75,
                    "solver": "forceAtlas2Based"
                }
            }
            """)
        
    output_file = "knowledge_graph.html"
    net.save_graph(output_file)
    print(f"Graph saved to {os.path.abspath(output_file)}")

    # Try to open in browser
    try:
        import webbrowser
        webbrowser.open(f"file://{os.path.abspath(output_file)}")
    except:
        print("Could not open browser automatically")
        
# Run the function
visualize_graph(graph_documents)

Graph saved to /home/sarsh/Knowledge_graph/knowledge_graph.html


Gtk-Message: 14:47:09.705: Not loading module "atk-bridge": The functionality is provided by GTK natively. Please try to not load it.
Gtk-Message: 14:47:09.775: Failed to load module "canberra-gtk-module"
Gtk-Message: 14:47:09.776: Failed to load module "canberra-gtk-module"


In [19]:
#Extract specific types of nodes
allowed_nodes = ["Person", "Organization", "Location", "Award", "ResearchField"]
graph_transformer_nodes_defined = LLMGraphTransformer(llm=llm, allowed_nodes=allowed_nodes)
graph_documents_nodes_defined = await graph_transformer_nodes_defined.aconvert_to_graph_documents(documents)

In [20]:
print(f"Nodes:{graph_documents_nodes_defined[0].nodes}")
print(f"Relationships:{graph_documents_nodes_defined[0].relationships}")

Nodes:[Node(id='1921 Nobel Prize in Physics', type='Award', properties={}), Node(id='Swiss Patent Office', type='Organization', properties={}), Node(id='unified field theory', type='ResearchField', properties={}), Node(id='American citizenship', type='Organization', properties={}), Node(id='Franklin D. Roosevelt', type='Person', properties={}), Node(id='theory of relativity', type='ResearchField', properties={}), Node(id='Satyendra Nath Bose', type='Person', properties={}), Node(id='quantum electrodynamics', type='ResearchField', properties={}), Node(id='Albert Einstein', type='Person', properties={})]
Relationships:[Relationship(source=Node(id='Albert Einstein', type='Person', properties={}), target=Node(id='theory of relativity', type='ResearchField', properties={}), type='DEVELOPED', properties={}), Relationship(source=Node(id='Albert Einstein', type='Person', properties={}), target=Node(id='1921 Nobel Prize in Physics', type='Award', properties={}), type='RECEIVED', properties={}),

In [22]:
#Extract specific types of relationships
allowed_nodes = ["Person", "Organization", "Location", "Award", "ResearchField"]
allowed_relationships = [
    ("Person", "WORKS_AT", "Organization"),
    ("Person", "SPOUSE", "Person"),
    ("Person", "AWARD", "Award"),
    ("Organization", "IN_LOCATION", "Location"),
    ("Person", "FIELD_OF_RESEARCH", "ResearchField")
]
graph_transformer_rel_defined = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=allowed_nodes,
    allowed_relationships=allowed_relationships
)
graph_documents_rel_defined = await graph_transformer_rel_defined.aconvert_to_graph_documents(documents)

In [23]:
#Visualize graph
visualize_graph(graph_documents_rel_defined)

Graph saved to /home/sarsh/Knowledge_graph/knowledge_graph.html


Gtk-Message: 15:02:39.214: Not loading module "atk-bridge": The functionality is provided by GTK natively. Please try to not load it.
Gtk-Message: 15:02:39.278: Failed to load module "canberra-gtk-module"
Gtk-Message: 15:02:39.279: Failed to load module "canberra-gtk-module"
